In [1]:
import pandas as pd
# import modin.pandas as mpd
import numpy as np
import datetime
from scipy.stats import norm
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import matplotlib.ticker as ticker
import mplfinance as mpf
from multiprocessing import Pool
from contextlib import contextmanager
from joblib import Parallel, delayed
from IPython.display import clear_output
# import dill as pickle
import pickle
import time

In [2]:
# causis api
from causis_api.const import get_version
from causis_api.const import login
login.username = 'shuai.song'
login.password = 'Tsinghua2022'
login.version = get_version()
from causis_api.data import *
from causis_api.tool import *

In [72]:
commodities = {
            'rb':{'multiplier':10,'mintick':1,'exchange':'SHF'},
            'hc':{'multiplier':10,'mintick':1,'exchange':'SHF'},
            'i':{'multiplier':100,'mintick':0.5,'exchange':'DCE'},
            'm':{'multiplier':10,'mintick':1,'exchange':'DCE'},
            'pp':{'multiplier':5,'mintick':1,'exchange':'DCE'},
            'MA':{'multiplier':10,'mintick':1,'exchange':'CZC'},
            'bu':{'multiplier':10,'mintick':2,'exchange':'SHF'},
            'l':{'multiplier':5,'mintick':5,'exchange':'DCE'},
            'p':{'multiplier':10,'mintick':2,'exchange':'DCE'},
            'v':{'multiplier':5,'mintick':5,'exchange':'DCE'},
            'CF':{'multiplier':5,'mintick':5,'exchange':'CZC'},
            'OI':{'multiplier':10,'mintick':1,'exchange':'CZC'},
            'SR':{'multiplier':10,'mintick':1,'exchange':'CZC'},
            'TA':{'multiplier':5,'mintick':2,'exchange':'CZC'},
            'SA':{'multiplier':20,'mintick':1,'exchange':'CZC'},
} 

In [75]:
commodities.keys()

(dict_keys(['rb', 'hc', 'i', 'm', 'pp', 'MA', 'bu', 'l', 'p', 'v', 'CF', 'OI', 'SR', 'TA', 'SA']),)

In [79]:
Periods = []
F_data = {}
for symbol in tqdm(commodities.keys(), desc='Data Accessing'):
    c = f"R.CN.{commodities[symbol]['exchange']}.{symbol}.0004"
    com = get_price(c, '2010-01-01', '2022-06-25', 'minute5')
    com[['OPEN', 'HIGH', 'CLOSE', 'LOW']] = com[['OPEN', 'HIGH', 'CLOSE', 'LOW']].astype('float64')
    com[['SYMBOL', 'CLOCK']] =  com[['SYMBOL', 'CLOCK']].astype('string')
    Periods.append((com['CLOCK'].iloc[0], com['CLOCK'].iloc[-1]))
    F_data[symbol] = com

Data Accessing: 100%|██████████| 15/15 [01:17<00:00,  5.19s/it]


,SYMBOL,CLOCK,OPEN,HIGH,LOW,CLOSE,VOLUME,INTEREST,SETTLE,ADJ
0,R.CN.SHF.rb.0004,2010-01-04 09:05:00,4523.0,4540.0,4523.0,4535.0,72858.0,919102.0,0.0,0.869811
1,R.CN.SHF.rb.0004,2010-01-04 09:10:00,4535.0,4536.0,4526.0,4530.0,40038.0,921780.0,0.0,0.869811
2,R.CN.SHF.rb.0004,2010-01-04 09:15:00,4529.0,4562.0,4529.0,4556.0,117404.0,937494.0,0.0,0.869811
3,R.CN.SHF.rb.0004,2010-01-04 09:20:00,4556.0,4560.0,4551.0,4558.0,50354.0,941688.0,0.0,0.869811
4,R.CN.SHF.rb.0004,2010-01-04 09:25:00,4558.0,4559.0,4553.0,4555.0,35572.0,934818.0,0.0,0.869811
...,...,...,...,...,...,...,...,...,...,...
177337,R.CN.SHF.rb.0004,2022-06-24 22:40:00,4216.0,4222.0,4210.0,4214.0,18448.0,1890113.0,0.0,1.245742
177338,R.CN.SHF.rb.0004,2022-06-24 22:45:00,4214.0,4233.0,4214.0,4231.0,45949.0,1892249.0,0.0,1.245742
177339,R.CN.SHF.rb.0004,2022-06-24 22:50:00,4232.0,4250.0,4227.0,4248.0,54663.0,1891120.0,0.0,1.245742
177340,R.CN.SHF.rb.0004,2022-06-24 22:55:00,4248.0,4281.0,4245.0,4247.0,101603.0,1885658.0,0.0,1.245742


In [45]:
c = 'R.CN.SHF.hc.0004'
slice = get_price(c, '2010-01-01', '2022-06-25', 'minute5')

In [65]:
slice[['OPEN', 'HIGH', 'CLOSE', 'LOW']] = slice[['OPEN', 'HIGH', 'CLOSE', 'LOW']].astype('float64')
slice[['SYMBOL', 'CLOCK']] =  slice[['SYMBOL', 'CLOCK']].astype('string')

In [46]:
list(slice['CLOCK'][:500])

['2014-03-24 09:05:00',
 '2014-03-24 09:10:00',
 '2014-03-24 09:15:00',
 '2014-03-24 09:20:00',
 '2014-03-24 09:25:00',
 '2014-03-24 09:30:00',
 '2014-03-24 09:35:00',
 '2014-03-24 09:40:00',
 '2014-03-24 09:45:00',
 '2014-03-24 09:50:00',
 '2014-03-24 09:55:00',
 '2014-03-24 10:00:00',
 '2014-03-24 10:05:00',
 '2014-03-24 10:10:00',
 '2014-03-24 10:15:00',
 '2014-03-24 10:35:00',
 '2014-03-24 10:40:00',
 '2014-03-24 10:45:00',
 '2014-03-24 10:50:00',
 '2014-03-24 10:55:00',
 '2014-03-24 11:00:00',
 '2014-03-24 11:05:00',
 '2014-03-24 11:10:00',
 '2014-03-24 11:15:00',
 '2014-03-24 11:20:00',
 '2014-03-24 11:25:00',
 '2014-03-24 11:30:00',
 '2014-03-24 13:35:00',
 '2014-03-24 13:40:00',
 '2014-03-24 13:45:00',
 '2014-03-24 13:50:00',
 '2014-03-24 13:55:00',
 '2014-03-24 14:00:00',
 '2014-03-24 14:05:00',
 '2014-03-24 14:10:00',
 '2014-03-24 14:15:00',
 '2014-03-24 14:20:00',
 '2014-03-24 14:25:00',
 '2014-03-24 14:30:00',
 '2014-03-24 14:35:00',
 '2014-03-24 14:40:00',
 '2014-03-24 14: